In [199]:
import pandas as pd
import numpy as np

df = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Cycle 5-8 Non DOB NOW Facade Filings (11-22-16).csv", low_memory=False )

df['Initial File Date Raw'] = df['Initial File Date Raw'].astype(str)
df['Initial File Date'] =  df['Initial File Date Raw'].apply(lambda x: x[4:6]) + "/" + df['Initial File Date Raw'].apply(lambda x: x[6:8]) + "/" + df['Initial File Date Raw'].apply(lambda x: x[:4])

df['Initial File Date'] = df['Initial File Date'].str.replace('//', '')

df['Initial File Year'] = df['Initial File Date Raw'].apply(lambda x: x[:4])

#df["BIN"] = df["BIN"].astype(int)

print "drop current status of NR"
print "original length of df", len(df)
df = df[df['Current Status'] != 'NR']
#df = df[df['BIN'] != ' ']

df = df.reset_index(drop=True)
print "new length of df", len(df)

print
print "drop rows with BIN = 0"
df = df[np.isfinite(df['BIN'])]
print "new length of df", len(df)
df["BIN"] = df["BIN"].astype(int)

df = df[["BIN", "Facade Cycle", "Number of Stories", "Initial Filing Status", "Current Status", \
         "Initial File Date", "Initial File Year", "Owner Name", "Owner Business Name", "Owner Address", "Owner City", "Owner State", \
         "Owner Type"]]

#drop NaN

print 
print "drop NaN"
print "length of df before drop", len(df)
df = df.dropna()
print "length of df after drop", len(df)

df = df.reset_index(drop=True)

#drop data before 2005
print
"""
print "drop all data before 2006"
print "length before drop", len(df)
print "drop nan first"
df = df[df['Initial File Year'] != 'nan']
print "length after drop nan", len(df)
df["Initial File Year"] = df["Initial File Year"].astype(int)
df = df[df["Initial File Year"] >= 2006]
print "length after drop year", len(df)
"""


df = df.reset_index(drop=True)


drop current status of NR
original length of df 51307
new length of df 38435

drop rows with BIN = 0
new length of df 38414

drop NaN
length of df before drop 38414
length of df after drop 37628



In [180]:
df = df.rename(columns={'Current Status': 'Status'})

In [ ]:
df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Cycle 5-8 Non DOB NOW Facade Filings (11-22-16) EDIT 3.csv")

In [ ]:
df.dtypes

In [ ]:
df.head()

In [200]:
df["Status"] =""

df = df.reset_index(drop=True)


#get set of bins
bin_list = []

for i in range(0, len(df)):
    bin_list.append(df["BIN"][i])
    
bin_set = set(bin_list)
bin_set = list(bin_set)
#bin_set

In [201]:
status_list = []

for i in range(0, len(df)):
    status_list.append(df["Initial Filing Status"][i])
    
status_set = set(status_list)
status_set = list(status_set)
status_set

['SW', 'UN', 'OK']

In [202]:
qn = pd.DataFrame(columns=('BIN','Status', 'Initial File Date'))


for i in range(0, len(bin_set)):

    if i%1000 == 0:
        print i

    df2 = df[df["BIN"] == bin_set[i]]
    df2 = df2.reset_index(drop=True)
    
    un = 0
    for j in range(0, len(df2)):
        if df2["Initial Filing Status"][j] == "UN":
            status = "UN"
            initial_file_date = df2["Initial File Date"][j]
            un = 1
        elif (df2["Initial Filing Status"][j] == "SW" and un == 0):
            status = "SW"
            initial_file_date = df2["Initial File Date"][j]

        else:
            status = df2["Initial Filing Status"][j]
            initial_file_date = df2["Initial File Date"][j]

    
    qn.loc[i] =  [bin_set[i],status, initial_file_date]

    #CALCULATE ANY TIME UN OR SW AT ANY TIME OF THE ENTIRE CYCLE HISTORY
    
    
qn["BIN"] = qn["BIN"].astype(int) 
qn["BIN"] = qn["BIN"].astype(str) 

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


In [ ]:
qn["Initial File Date"][2610]

# MERGE WITH PLUTO DATA BASED ON BEN'S DATATASET

In [204]:
#ben dataset has the building information, the qn dataset has the correct "safe" or "unsafe" data



dfb = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Cycle 5-8 Non DOB NOW Facade Filings (11-22-16) EDIT 2_PLUTO.csv", low_memory=False )

dfb["BIN"] = dfb["BIN"].astype(str)
df["BIN"] = df["BIN"].astype(str)

#qn = df

merged = pd.merge(qn, dfb)

df = merged


co = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\City Owned Buildings.csv")

rs = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Rent Stabilized Buildings 12.22.16_BIN.csv")

#code if publically owned
pd.set_option('chained_assignment', None)


df["Public"] = ""
df["Rent Stabalized"] = ""

df["BIN"] = df["BIN"].astype(str)

#create publically owned building set
#get set of bins
bin_pub = []

for i in range(0, len(co)):
    bin_pub.append(co["BIN"][i])
    
bin_pub = set(bin_pub)
bin_pub = list(bin_pub)
#bin_set

#create rent stabalied bin set
bin_rs = []
for i in range(0, len(rs)):
    bin_rs.append(rs["BIN"][i])
    
bin_rs = set(bin_rs)
bin_rs = list(bin_rs)

count = 0
for i in range(0, len(df)):
    if df["BIN"][i] in bin_pub:
        df["Public"][i] = 1
        #print "in here"
        count+=1
    else:
        df["Public"][i] = 0

countrs = 0
for i in range(0, len(df)):
    if df["BIN"][i] in bin_rs:
        df["Rent Stabalized"][i] = 1
        countrs+=1
    else:
        df["Rent Stabalized"][i] = 0

print "length of df before dropping zeros", len(df) 
before  = len(df)

#calculate age
df["Age"] = 2016 - df["YearBuilt"]


df = df[df["NumFloors"] != 0]
df = df[df["NumFloors_DOITT"] != 0]
df = df[df["YearBuilt"] != 0]
df = df[df["Age"] >= 0]
#df = df[df["BldgFront"] != 0]

df = df.reset_index(drop=True)


#aggregate building class
rm_list = ['RA', 'RB', 'RC', 'RD', 'RH', 'RI', 'RK', 'RM', 'RR', 'RX', 'RZ']

df["BldgClassAgg"] = ""

for i in range(0, len(df)):
    if df["BldgClass"][i] not in rm_list:
        w = list(df["BldgClass"][i])
        df["BldgClassAgg"][i] = w[0]
    else:
        df["BldgClassAgg"][i] = df["BldgClass"][i]

df["BldgArea"] = df["BldgFront"]*df["BldgDepth"]
        
for i in range(0, len(df)):
    if df["BldgFront"][i] == 0:
        df["BldgFront"][i] = ""
    if df["BldgDepth"][i] == 0:
        df["BldgDepth"][i] = ""
    if df["BldgArea"][i] == 0:
        df["BldgArea"][i] = ""  

#categorize building unsafe/safe
df["Status"] = df["Status"].astype(str)

df["Status"] = df["Status"].str.replace('UN', 'UNSAFE')
df["Status"] = df["Status"].str.replace('SW', 'SAFE')
df["Status"] = df["Status"].str.replace('OK', 'SAFE')
#df["Status"] = df["Status"].astype(int)

#Landmark
df["Landmark"] = df["Landmark"].map(str.strip)
df["Landmark2"] = ""
landmark = 0
for i in range(0, len(df)):
    s = df["Landmark"][i]
    if len(s) > 1:
        df["Landmark2"][i] = 1
        landmark+=1
    else:
        df["Landmark2"][i] = 0

#put assessed value in millions of dollars
df["AssessTotal"] =  df["AssessTotal"]/1000000
        
print "count of public buildings", count
print "count of rent stabalized buildings", countrs
print "number of observations in df after dropping", len(df)
print "difference", (before  - len(df))
print "number of landmark buildings", landmark

length of df before dropping zeros 12789
count of public buildings 1785
count of rent stabalized buildings 4748
number of observations in df after dropping 12664
difference 125
number of landmark buildings 301


In [205]:
#Add Facade violations
fv = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Failure to maintain_protect (All years, all classes, REVISED).csv", low_memory=False )


fv["Vio Issued Date"] = pd.to_datetime(fv["Vio Issued Date"])

df["Initial File Date"] = pd.to_datetime(df["Initial File Date"])

dc = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Facade, Brickwork, Exterior Wall Complaints.csv", low_memory=False )

dc["Date Entered"] = pd.to_datetime(dc["Date Entered"])


In [ ]:
fv["BIN"] = fv["BIN"].astype(int)
fv["BIN"] = fv["BIN"].astype(str)

dc["BIN Number"] = dc["BIN Number"].astype(str)

df["BIN"] = df["BIN"].astype(str)

#drop the row with Nan for date
df = df.drop(df.index[[2611]])
df = df.reset_index(drop=True)

df["Violations"] = 0
df["Complaints"] = 0

for i in range(0, len(df)):
    #violations
    fv2 = fv[fv["BIN"] == df["BIN"][i]]
    fv2 = fv2.reset_index(drop=True)
    
    initial_date = str(df["Initial File Date"][i].month) + '-' + str(df["Initial File Date"][i].day) + '-' + str(df["Initial File Date"][i].year)
    
 
    #print "i=", i
    #print "initial date", initial_date
    mask = (fv['Vio Issued Date'] < initial_date)

    fv2 = fv2.loc[mask] 
    #fv2 = fv2.reset_index(drop=True)
        
    df["Violations"][i] = len(fv2)
    
    
    #complaints
    dc2 = dc[dc["BIN Number"] == df["BIN"][i]]
    dc2 = dc2.reset_index(drop=True)
    
    #mask = (dc['Date Entered'] < initial_date)
    
    #dc2 = dc2.loc[mask] 

    df["Complaints"][i] = len(dc2)
 
    
    # if i%2 == 0:
    #    print i
    #    print initial_date
    
    if i%500 == 0:
        print "i=", i
        print len(fv2)

In [207]:
#Add wall surface area

dws =  pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\WallSurfaceArea.csv", low_memory=False )

dws = dws[["bin", "WallArea"]]

dws["bin"] = dws["bin"].astype(str)

dwsG = (dws.groupby('bin').agg({'WallArea':'sum'}) )

dwsG = dwsG.reset_index()

dwsDic = dwsG.set_index('bin').to_dict()

dwsDic = dwsG.set_index('bin')['WallArea'].to_dict()

df["WallSurfaceArea"] = ""

for i in range(0, len(df)):
    if df["BIN"][i] in dwsDic:
        df["WallSurfaceArea"][i] = dwsDic[df["BIN"][i]]




In [208]:
print "remove floors less than six"
print "original dataframe size", len(df)
df = df[df["NumFloors"] >= 6]
print "dataframe size", len(df)



remove floors less than six
original dataframe size 12663
dataframe size 12491


In [41]:
print len(dwsG)

20896


In [ ]:
df["Initial File Date"][2611]

In [ ]:
df.dtypes

In [ ]:
df.head(2)

In [ ]:
fv.head(2)

In [118]:
#EXPORT ONLY PUBLIC BUILDINGS
#export = df
#export = df[df["Public"] == 1]
#export.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\FACADE ANALYSIS DEC TREE 5 PUBLIC.csv")

In [191]:
df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\FACADE ANALYSIS DEC TREE 6 CURRENT STATUS.csv")

In [ ]:
df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\test.csv")

In [ ]:
df.describe().transpose()

In [ ]:
df.dtypes

In [ ]:
qn.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\test.csv")

In [ ]:
df.dtypes

In [ ]:
#df = df[["Facade Cycle", ""]]

In [ ]:
df

# RECODE MAIN VARIABLES OF INTEREST

In [193]:
df = df.reset_index(drop=True)



df["YearBuiltCategory"] = ""
df["AssessTotalCategory"] = ""
df["NumFloorsCategory"] = ""

for i in range (0, len(df)):
    if df["YearBuilt"][i] < 1900:
        df["YearBuiltCategory"][i] = "pre-1900"
        
    if (df["YearBuilt"][i] >= 1900 and df["YearBuilt"][i] < 1916):
        df["YearBuiltCategory"][i] = "1900-1915"
        
    if (df["YearBuilt"][i] >= 1916 and df["YearBuilt"][i] < 1938):
        df["YearBuiltCategory"][i] = "1916-1937"
        
    if (df["YearBuilt"][i] >= 1938 and df["YearBuilt"][i] < 1968):
        df["YearBuiltCategory"][i] = "1938-1967"
        
    if (df["YearBuilt"][i] >= 1968 and df["YearBuilt"][i] < 2008):
        df["YearBuiltCategory"][i] = "1968-2007"
        
    if (df["YearBuilt"][i] >= 2008 and df["YearBuilt"][i] < 2017):
        df["YearBuiltCategory"][i] = "2008-2016"
        
    
    if df["AssessTotal"][i] < 10:
        df["AssessTotalCategory"][i] = "Low Value"
        
    if (df["AssessTotal"][i] >= 10 and df["AssessTotal"][i] < 20):
        df["AssessTotalCategory"][i] = "Medium Value"
        
    if df["AssessTotal"][i] >= 20:
        df["AssessTotalCategory"][i] = "High Value"
        
        
    if df["NumFloors"][i] < 10:
        df["NumFloorsCategory"][i] = "Low Rise"
        
    if df["NumFloors"][i] >= 10:
        df["NumFloorsCategory"][i] = "High Rise"
        
  
    
    

In [196]:
#subset for facade cycle, want all of cycle 7, last complete cycle
df["Facade Cycle"] = df["Facade Cycle"].astype(str)

print "length before drop", len(df)

df = df[df["Facade Cycle"] != '5']
df = df[df["Facade Cycle"] != '6']
df = df[df["Facade Cycle"] != '8A']
df = df[df["Facade Cycle"] != '8B']
df = df[df["Facade Cycle"] != '8C']

df = df.reset_index(drop=True)

print "length after drop", len(df)

length before drop 21464
length after drop 9830


In [198]:
df.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\FACADE ANALYSIS DEC TREE 6 CURRENT STATUS.csv")

# Code PLUTO Data for Status, Complaints and Public

In [209]:
dfpluto = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\PLUTO\\MapPLUTO_BINs_1toManyJOIN.csv", low_memory=False )

print "remove floors less than six"
print "original dataframe size", len(dfpluto)
dfpluto = dfpluto[dfpluto["NumFloors"] >= 6]
print "dataframe size", len(dfpluto)
dfpluto = dfpluto.reset_index(drop=True)


remove floors less than six
original dataframe size 1124645
dataframe size 28597


In [210]:
dfpluto = dfpluto.reset_index(drop=True)
df = df.reset_index(drop=True)


dfpluto["Status"] = ""
#dfpluto["Complaint"] = 0
#dfpluto["Public"] = 0

dfpluto["bin"] = dfpluto["bin"].astype(int)
dfpluto["bin"] = dfpluto["bin"].astype(str)


dfunDic = df.set_index('BIN')['Status'].to_dict()

count=0
for i in range(0, len(dfpluto)):
    if dfpluto["bin"][i] in dfunDic:
        count+=1
        dfpluto["Status"][i] = dfunDic[dfpluto["bin"][i]]

print "count of buildings labeled unsafe or safe", count


"""
#complaints
count = 0
dfcDic = df.set_index('BIN')['Complaints'].to_dict()
for i in range(0, len(dfpluto)):
    if dfpluto["bin"][i] in dfcDic:
        count+=1
        dfpluto["Complaint"][i] = dfcDic[dfpluto["bin"][i]]

print "count of buildings with complaints", count



print "total number of complaints in pluto", count

bin_pub

#public
count=0
for i in range(0, len(dfpluto)):
    if dfpluto["bin"][i] in bin_pub:
        count+=1
        dfpluto["Public"][i] = 1
print "total number of public b in pluto", count
"""

count of buildings labeled unsafe or safe 12082


'\n#complaints\ncount = 0\ndfcDic = df.set_index(\'BIN\')[\'Complaints\'].to_dict()\nfor i in range(0, len(dfpluto)):\n    if dfpluto["bin"][i] in dfcDic:\n        count+=1\n        dfpluto["Complaint"][i] = dfcDic[dfpluto["bin"][i]]\n\nprint "count of buildings with complaints", count\n\n\n\nprint "total number of complaints in pluto", count\n\nbin_pub\n\n#public\ncount=0\nfor i in range(0, len(dfpluto)):\n    if dfpluto["bin"][i] in bin_pub:\n        count+=1\n        dfpluto["Public"][i] = 1\nprint "total number of public b in pluto", count\n'

In [211]:
#subset data
#export = dfpluto[["bin", "Address", "Borough", "OwnerName", "NumFloors", "num_floors", "BldgFront", "AssessTot", "YearBuilt","Status", "Complaint", "Public", "XCoord", "YCoord"]]
export = dfpluto[["bin", "Address", "Borough", "OwnerName", "NumFloors", "num_floors", "BldgFront", "AssessTot", "YearBuilt","Status", "XCoord", "YCoord"]]

export["Age"] = 2016 - export["YearBuilt"]
export["AssessTot"] = export["AssessTot"].astype(float)
export["AssesTot(Million)"] = export["AssessTot"]/1000000.0

In [164]:
#Recode Pluto data
export["YearBuiltCategory"] = ""
export["AssessTotalCategory"] = ""
export["NumFloorsCategory"] = ""

for i in range (0, len(export)):
    if export["YearBuilt"][i] < 1900:
        export["YearBuiltCategory"][i] = "pre-1900"
        
    if (export["YearBuilt"][i] >= 1900 and export["YearBuilt"][i] < 1916):
        export["YearBuiltCategory"][i] = "1900-1915"
        
    if (export["YearBuilt"][i] >= 1916 and export["YearBuilt"][i] < 1938):
        export["YearBuiltCategory"][i] = "1916-1937"
        
    if (export["YearBuilt"][i] >= 1938 and export["YearBuilt"][i] < 1968):
        export["YearBuiltCategory"][i] = "1938-1967"
        
    if (export["YearBuilt"][i] >= 1968 and export["YearBuilt"][i] < 2008):
        export["YearBuiltCategory"][i] = "1968-2007"
        
    if (export["YearBuilt"][i] >= 2008 and export["YearBuilt"][i] < 2017):
        export["YearBuiltCategory"][i] = "2008-2016"
        
    
    if export["AssesTot(Million)"][i] < 10:
        export["AssessTotalCategory"][i] = "Low Value"
        
    if (export["AssesTot(Million)"][i] >= 10 and export["AssesTot(Million)"][i] < 20):
        export["AssessTotalCategory"][i] = "Medium Value"
        
    if export["AssesTot(Million)"][i] >= 20:
        export["AssessTotalCategory"][i] = "High Value"
        
        
    if export["NumFloors"][i] < 10:
        export["NumFloorsCategory"][i] = "Low Rise"
        
    if export["NumFloors"][i] >= 10:
        export["NumFloorsCategory"][i] = "High Rise"
        


In [166]:
#DROP STATUS = NA
print "length befor drop", len(export)
export = export[export["Status"] != ""]
print "length after drop", len(export)

#print "drop owner name = NA"


length befor drop 28597
length after drop 12199


In [90]:
#export pluto validation dataset
#dfpluto.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\PLUTO VALIDATION DATA.csv")

In [213]:
export.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\PLUTO VALIDATION DATA 2.csv")

# PLUTO SUMMARY STATS ON SAFE/UNSAFE BUILDINGS

In [138]:
exportUN  = export[export['Status'] == 'UNSAFE']
print len(exportUN)


2459


In [139]:
exportUN.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NumFloors,2459.0,1.227410e+01,7.537194e+00,6.00000,7.000000e+00,1.000000e+01,1.500000e+01,8.800000e+01
num_floors,2459.0,1.226881e+01,7.537385e+00,6.00000,7.000000e+00,1.000000e+01,1.500000e+01,8.800000e+01
BldgFront,2459.0,1.286048e+02,1.238786e+02,0.00000,5.000000e+01,1.000000e+02,1.665000e+02,1.160000e+03
AssessTot,2459.0,1.700808e+07,3.144390e+07,204750.00000,3.205125e+06,9.031050e+06,1.727078e+07,3.834945e+08
YearBuilt,2459.0,1.944390e+03,2.427285e+01,1836.00000,1.925000e+03,1.950000e+03,1.963000e+03,2.014000e+03
Complaint,2459.0,9.150061e-01,8.146433e-01,0.00000,0.000000e+00,1.000000e+00,1.000000e+00,7.000000e+00
Public,2459.0,3.574624e-01,4.793501e-01,0.00000,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
XCoord,2459.0,9.995872e+05,1.606151e+04,937004.00000,9.885355e+05,9.969250e+05,1.006946e+06,1.064964e+06
YCoord,2459.0,2.120335e+05,2.615576e+04,147779.00000,1.965280e+05,2.131010e+05,2.310925e+05,2.705570e+05
Age,2459.0,7.160960e+01,2.427285e+01,2.00000,5.300000e+01,6.600000e+01,9.100000e+01,1.800000e+02


In [140]:
exportS  = export[export['Status'] == 'SAFE']
print len(exportS)


9740


In [141]:
exportS.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NumFloors,9740.0,1.222387e+01,8.414880e+00,6.000000,6.000000e+00,9.000000e+00,1.500000e+01,8.500000e+01
num_floors,9740.0,1.221417e+01,8.419895e+00,0.000000,6.000000e+00,9.000000e+00,1.500000e+01,8.500000e+01
BldgFront,9740.0,1.171782e+02,1.624570e+02,0.000000,5.000000e+01,9.700000e+01,1.540000e+02,7.538000e+03
AssessTot,9740.0,2.155496e+07,5.553119e+07,27093.000000,2.579850e+06,6.547275e+06,1.719405e+07,9.069332e+08
YearBuilt,9740.0,1.943546e+03,2.787146e+01,1846.000000,1.922000e+03,1.940000e+03,1.962000e+03,2.015000e+03
Complaint,9740.0,1.991786e-01,5.326966e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e+00
Public,9740.0,8.367556e-02,2.769149e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
XCoord,9740.0,9.971551e+05,3.971499e+04,0.000000,9.882332e+05,9.937120e+05,1.004653e+06,1.065288e+06
YCoord,9740.0,2.147613e+05,2.335464e+04,0.000000,2.047928e+05,2.147150e+05,2.266412e+05,2.721550e+05
Age,9740.0,7.245370e+01,2.787146e+01,1.000000,5.400000e+01,7.600000e+01,9.400000e+01,1.700000e+02


In [ ]:
bldgclass = df.BldgClass.unique()

In [ ]:
print sorted(bldgclass)

# SUBSET DATA USING SAFE RULES FROM MODEL

In [215]:
export.head()

,bin,Address,Borough,OwnerName,NumFloors,num_floors,BldgFront,AssessTot,YearBuilt,Status,XCoord,YCoord,Age,AssesTot(Million)
0,2014444,2501 DAVIDSON AVENUE,BX,DAVIDSON APARTMENTSLL,6.0,6.0,175.0,1182600.0,1935,,1011578,254280,81,1.18260
1,2008116,1454 WALTON AVENUE,BX,WALTON AVE ASSOCIATES,6.0,6.0,175.0,1742850.0,1928,,1007845,245617,88,1.74285
2,2003075,1190 SHAKESPEARE AVENUE,BX,1190 SHAKESPEARE LLC.,6.0,6.0,203.0,1413900.0,1929,SAFE,1005540,243978,87,1.41390
3,2083276,629 KAPPOCK STREET,BX,BONNIE CREST OWNERS C,6.0,6.0,215.0,3228750.0,1961,SAFE,1007172,260009,55,3.22875
4,2048824,2132 WALLACE AVENUE,BX,2132 PRESIDENTIAL ASS,6.0,6.0,200.0,3787200.0,1929,,1021774,250454,87,3.78720


In [260]:
#Age <= 33: SAFE (927.6/216.6)


ageless33 = export[export["Age"] <= 33]
print len(ageless33)


print "accuracy"
print "safe"
statusS = ageless33[ageless33["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = ageless33[ageless33["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)


5162
accuracy
safe
1043
unsafe
103
percent 0.1


In [259]:
#Age > 33: :...AssessTotal > 383.4945: SAFE (32.7)
assGreat383 = export[export["Age"] > 33]
assGreat383 = assGreat383[assGreat383["AssesTot(Million)"] > 383.4945]

print len(assGreat383)
print "accuracy"
print "safe"
statusS = assGreat383[assGreat383["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assGreat383[assGreat383["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)

200
accuracy
safe
54
unsafe
0
percent 0.0


In [258]:
#   Age > 33: AssessTotal <= 383.4945: :...AssessTotal > 46.01025: :...Age > 66: SAFE (357.5/90.1)

assLess383 = export[export["Age"] > 33]
assLess383 = assLess383[assLess383["AssesTot(Million)"] <= 383.4945]
assLess383 = assLess383[assLess383["AssesTot(Million)"] > 46.01025]
assLess383 = assLess383[assLess383["Age"] > 66]

print len(assLess383)

print "accuracy"
print "safe"
statusS = assLess383[assLess383["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assLess383[assLess383["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)

753
accuracy
safe
403
unsafe
52
percent 0.13


In [257]:
#   Age > 33: AssessTotal <= 383.4945: :...AssessTotal > 46.01025: :...Age <= 66: SAFE (357.5/90.1) NumFloors > 18: SAFE (251.1/69.5)

assLess383NF = export[export["Age"] > 33]
assLess383NF = assLess383NF[assLess383NF["AssesTot(Million)"] <= 383.4945]
assLess383NF = assLess383NF[assLess383NF["AssesTot(Million)"] > 46.01025]
assLess383NF = assLess383NF[assLess383NF["Age"] <= 66]
assLess383NF = assLess383NF[assLess383NF["NumFloors"] > 18]

print len(assLess383NF)
print "accuracy"
print "safe"
statusS = assLess383NF[assLess383NF["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assLess383NF[assLess383NF["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)

555
accuracy
safe
265
unsafe
35
percent 0.13


In [228]:
assLess383NF = assLess383NF.reset_index(drop=True)
#for i in range(0, len(assLess383NF)):
#    print assLess383NF["Status"][i]

In [253]:
#   Age > 33: AssessTotal <= 383.4945: :...AssessTotal <= 46.01025: BldgFront <= 698: :...NumFloors > 6: :...BldgFront <= 100.92: SAFE 




assLess46a = export[export["Age"] > 33]
assLess46a = assLess46a[assLess46a["AssesTot(Million)"] <= 383.4945]
assLess46a = assLess46a[assLess46a["AssesTot(Million)"] <= 46.01025]
assLess46a = assLess46a[assLess46a["BldgFront"] <= 698]
assLess46a = assLess46a[assLess46a["NumFloors"] > 6]
assLess46a = assLess46a[assLess46a["BldgFront"] <= 100.92]


print len(assLess46a)

print "accuracy"
print "safe"
statusS = assLess46a[assLess46a["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assLess46a[assLess46a["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)

5235
accuracy
safe
3387
unsafe
877
percent 0.26


In [256]:

#   Age > 33: AssessTotal <= 383.4945: :...AssessTotal <= 46.01025: BldgFront <= 698: :...NumFloors > 6: :...BldgFront > 100.92: 
# Age <= 37: SAFE (55.4/15.7)
assLess46 = export[export["Age"] > 33]
assLess46 = assLess46[assLess46["AssesTot(Million)"] <= 383.4945]
assLess46 = assLess46[assLess46["AssesTot(Million)"] <= 46.01025]
assLess46 = assLess46[assLess46["BldgFront"] <= 698]
assLess46 = assLess46[assLess46["NumFloors"] > 6]
assLess46 = assLess46[assLess46["BldgFront"] > 100.92]
assLess46 = assLess46[assLess46["Age"] < 37]



print len(assLess46)

print "accuracy"
print "safe"
statusS = assLess46[assLess46["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assLess46[assLess46["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)


46
accuracy
safe
31
unsafe
6
percent 0.19


In [231]:
assLess46 = assLess46.reset_index(drop=True)
#for i in range(0, len(assLess46)):
#    print assLess46["Status"][i]

In [254]:
#   Age > 33: AssessTotal <= 383.4945: :...AssessTotal <= 46.01025: BldgFront <= 698: :...NumFloors <= 6:
# Age > 50: :...AssessTotal <= 12.8151: SAFE (2622.7/965.3)
assLess12 = export[export["Age"] > 33]
assLess12 = assLess12[assLess12["AssesTot(Million)"] <= 383.4945]
assLess12 = assLess12[assLess12["AssesTot(Million)"] <= 46.01025]
assLess12 = assLess12[assLess12["BldgFront"] <= 698]
assLess12 = assLess12[assLess12["NumFloors"] == 6]
assLess12 = assLess12[assLess12["Age"] > 50]
assLess12 = assLess12[assLess12["AssesTot(Million)"] <= 12.8151]

print len(assLess12)


print "accuracy"
print "safe"
statusS = assLess12[assLess12["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assLess12[assLess12["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)




11907
accuracy
safe
2426
unsafe
423
percent 0.17


In [255]:
#   Age > 33: AssessTotal <= 383.4945: :...AssessTotal <= 46.01025: BldgFront <= 698: :...NumFloors <= 6:
# Age > 50: :...AssessTotal > 12.8151: AssessTotal > 16.52985 SAFE (2622.7/965.3)(138.8/48.8)

assGreat12 = export[export["Age"] > 33]
assGreat12 = assGreat12[assGreat12["AssesTot(Million)"] <= 383.4945]
assGreat12 = assGreat12[assGreat12["AssesTot(Million)"] <= 46.01025]
assGreat12 = assGreat12[assGreat12["BldgFront"] <= 698]
assGreat12 = assGreat12[assGreat12["NumFloors"] == 6]
assGreat12 = assGreat12[assGreat12["Age"] > 50]
assGreat12 = assGreat12[assGreat12["AssesTot(Million)"] > 16.52985]

print len(assGreat12)

print "accuracy"
print "safe"
statusS = assGreat12[assGreat12["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = assGreat12[assGreat12["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)


298
accuracy
safe
100
unsafe
35
percent 0.35


In [265]:
#merge all of the dataframes

frames = [ageless33,assGreat383, assLess383, assLess383NF, assLess46a, assLess46, assLess12, assGreat12]

result = pd.concat(frames)

result = result.reset_index(drop=True)

In [276]:
result["bin"] = result["bin"].astype(str)

#get set of bins
r_list = []

for i in range(0, len(result)):
    r_list.append(result["bin"][i])
    
r_set = set(r_list)
r_set = list(r_set)

print len(r_set)

22531


In [277]:
max(r_set)

'5164479'

In [274]:
print len(export)
print len(result)

28597
24156


In [280]:
final_result = export

for i in range(0, len(export)):
    if i%1000 == 0:
        print i
    if export["bin"][i] in r_set:
        final_result  = final_result[final_result['bin'] != export["bin"][i]]

print len(final_result)

4291


In [282]:
print "accuracy"
print "safe"
statusS = final_result[final_result["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusSU = final_result[final_result["Status"] == "UNSAFE"]
print len(statusSU)

print "percent",  round(len(statusSU)/float(len(statusS) ), 2)

accuracy
safe
1894
unsafe
914
percent 0.48


In [304]:
final_result2 = final_result
final_result2 = final_result2[final_result2["Status"] != "SAFE"]
final_result2 = final_result2[final_result2["Status"] != "UNSAFE"]

print len(final_result2)

#sort by values know have predictive value

final_result2 = final_result2.sort_values(['AssesTot(Million)', 'BldgFront', 'NumFloors', 'Age'], ascending=[True, False, False, False])

final_result2 = final_result2.reset_index(drop=True)



1483


In [314]:
#validate that the building is unknown to the dob

final_result2["Known to DOB"] = "No"

val = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Cycle 5-8 Non DOB NOW Facade Filings (11-22-16).csv", low_memory=False )

val = val[np.isfinite(val["BIN"])]
val = val.reset_index(drop=True)

val["BIN"] = val["BIN"].astype(int)
val["BIN"] = val["BIN"].astype(str)




#get set of bins
val_list = []

for i in range(0, len(val)):
    val_list.append(val["BIN"][i])
    
val_set = set(val_list)
val_set = list(val_set)

print len(val_set)

count=0

for i in range(0, len(final_result2)):
    if final_result2["bin"][i] in val_set:
        final_result2["Known to DOB"][i] = "Yes"
        count+=1
        
print count



14178
59


In [320]:
#drop duplicates
final_result2 = final_result2.drop_duplicates(['Address']).reset_index(drop=True)

print len(final_result2)

705


In [321]:
final_result2.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\FINAL INITIAL RESULTS 1.csv")

In [317]:
len(final_result2)

1483

# SUBSET DATA USING UNSAFE RULES FROM MODEL

In [287]:
#Age > 33: AssessTotal <= 383.4945: :...AssessTotal > 46.01025: :  Age <= 66:   :...NumFloors <= 18: UNSAFE (153.3/53.4)
assLess383NF = export[export["Age"] > 33]
assLess383NF = assLess383NF[assLess383NF["AssesTot(Million)"] <= 383.4945]
assLess383NF = assLess383NF[assLess383NF["AssesTot(Million)"] > 46.01025]
assLess383NF = assLess383NF[assLess383NF["Age"] <= 66]
assLess383NF = assLess383NF[assLess383NF["NumFloors"] <= 18]

print len(assLess383NF)
print "accuracy"
print "safe"
statusS = assLess383NF[assLess383NF["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusU = assLess383NF[assLess383NF["Status"] == "UNSAFE"]
print len(statusSU)

print "percent unsafe",  round(len(statusU)/   float(len(statusS) + len(statusU)      ), 2)

374
accuracy
safe
81
unsafe
56
percent unsafe 0.41


In [289]:
#Age > 33: AssessTotal <= 383.4945: :...AssessTotal <= 46.01025: : BldgFront <= 698: 

assLess383NF = export[export["Age"] > 33]
assLess383NF = assLess383NF[assLess383NF["AssesTot(Million)"] <= 383.4945]
assLess383NF = assLess383NF[assLess383NF["AssesTot(Million)"] <= 46.01025]
assLess383NF = assLess383NF[assLess383NF["BldgFront"] > 698]

print len(assLess383NF)
print "accuracy"
print "safe"
statusS = assLess383NF[assLess383NF["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusU = assLess383NF[assLess383NF["Status"] == "UNSAFE"]
print len(statusSU)

print "percent unsafe",  round(len(statusU)/   float(len(statusS) + len(statusU)      ), 2)


56
accuracy
safe
3
unsafe
56
percent unsafe 0.92


In [290]:
#Age > 33: AssessTotal <= 383.4945: :...AssessTotal <= 46.01025: : BldgFront <= 698: NumFloors > 6:  BldgFront > 100.92:Age > 37: UNSAFE (2695.9/1336.5)
unsafeAge = export[export["Age"] > 33]

unsafeAge = unsafeAge[unsafeAge["AssesTot(Million)"] <= 383.4945]
unsafeAge = unsafeAge[unsafeAge["AssesTot(Million)"] <= 46.01025]
unsafeAge = unsafeAge[unsafeAge["BldgFront"] <= 698]
unsafeAge = unsafeAge[unsafeAge["NumFloors"] > 6]
unsafeAge = unsafeAge[unsafeAge["BldgFront"] > 100.92]
unsafeAge = unsafeAge[unsafeAge["Age"] > 37]

print len(unsafeAge)
print "accuracy"
print "safe"
statusS = unsafeAge[unsafeAge["Status"] == "SAFE"]
print len(statusS)

print "unsafe"
statusU = unsafeAge[unsafeAge["Status"] == "UNSAFE"]
print len(statusSU)

print "percent unsafe",  round(len(statusU)/   float(len(statusS) + len(statusU)      ), 2)



3285
accuracy
safe
1653
unsafe
56
percent unsafe 0.3
